# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is a very simple parts model.

    package 'Simple Action Example' {
    import Base::Anything;
    import ScalarValues::*;
    package 'Fake Library' {
        action def Action {
            action subactions : Action [0..20];
        }
        connection def Connection {
            end source: Anything;
            end target: Anything;
        }
        connection def HappensDuring :> Connection {
            end earlierOccurrence: Action :>> Connection::source;
            end laterOccurrence: Action :>> Connection::target;
        }
    }
    
   
    
    action 'Build Burger': 'Fake Library'::Action {
        attribute 'Burger Kind' : String;
        first start;
        then action 'Place Buns' : 'Fake Library'::Action;
        then action 'Add Patty' : 'Fake Library'::Action;
        
        decide 'Next Topping';
        if 'Burger Kind' == "Cheeseburger" then 'Add Cheese';
        if 'Burger Kind' == "Hamburger" then 'Dress Burger';
        
        action 'Add Cheese' : 'Fake Library'::Action [0..1];
        action 'Dress Burger' : 'Fake Library'::Action [0..1];
        
        merge 'Finish Build';
        
        action 'Plate Burger' : 'Fake Library'::Action;
        then done;
        
        succession pattyToTopping : 'Fake Library'::HappensDuring first 'Add Patty' then 'Next Topping';
        succession cheeseToFinish : 'Fake Library'::HappensDuring first 'Add Cheese' then 'Finish Build';
        succession dressToFinish : 'Fake Library'::HappensDuring first 'Dress Burger' then 'Finish Build';
        
        succession 'Finish Build' then 'Plate Burger';
    }
}

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [ ]:
import os
from pathlib import Path

import networkx as nx

import pymbe.api as pm
from pymbe.client import SysML2Client
from pymbe.label import *
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.calc_dependencies import generate_execution_order, generate_parameter_signature_map
from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.interpretation import repack_instance_dictionaries
from pymbe.interpretation.results import *

from pymbe.local.stablization import build_stable_id_lookups

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [ ]:
helper_client = SysML2Client()

elements_data_path = Path("../../..") / "tests" / "fixtures" / "Simple Action Example.json"
assert elements_data_path.exists(), f"Could not find: '{elements_data_path}'"

helper_client._load_from_file(elements_data_path)

Create a graph representation of the model and load it into memory.

In [ ]:
lpg = SysML2LabeledPropertyGraph()
lpg.model = helper_client.model

This is just a helper to make abbreviations more legible.

In [ ]:
shorten_pre_bake = {
}

## Key IDs

The unique identifiers below are useful references for walking through the interpretations generated in this notebook.

In [ ]:
eig = lpg.get_projection("Expression Inferred")

execution_pairs = []
execution_contexts = {}

roots = [lpg.model.elements[node] for node in eig.nodes if eig.in_degree(node) == 0]

In [ ]:
roots

In [ ]:
[root.featuringType for root in roots]

In [ ]:
name_to_id_lookup = build_stable_id_lookups(lpg)[1]
name_to_id_lookup

In [ ]:
SIMPLE_ACTION_EXAMPLE = 'Model::Simple Action Example::'
FAKE_LIBRARY = 'Model::Simple Action Example::Fake Library::'

add_cheese_action_id = name_to_id_lookup[f'{SIMPLE_ACTION_EXAMPLE}Build Burger: Action <<ActionUsage>>']
action_id = name_to_id_lookup[f'{FAKE_LIBRARY}Action <<ActionDefinition>>']

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [ ]:
from pymbe.query.query import get_types_for_feature, get_features_typed_by_type, roll_up_multiplicity_for_type

In [ ]:
get_types_for_feature(lpg, add_cheese_action_id)

In [ ]:
get_features_typed_by_type(lpg, action_id)

In [ ]:
banded_featuring_graph = lpg.get_projection("Expanded Banded")

In [ ]:
banded_roots = [
        lpg.model.elements[node]
        for node in banded_featuring_graph.nodes
        if banded_featuring_graph.out_degree(node) < 1
    ]
banded_roots

In [ ]:
m0_interpretation = random_generator_playbook(
    lpg,
    shorten_pre_bake,
)